## plot

### 新版画图

In [ ]:
import os

result_dir = './final_plot/sampling_raw/result_all_models_fixstereoTautomer_filtered_trainset/'
save_dir = './final_plot/sampling_raw/result_all_models_fixstereoTautomer_filtered_trainset/figs_merge/'

denovo_uniprot_list = []
hit2lead_uniprot_list = []
for type_name in ['Denovo_Results','Hit_to_Lead_Results']:
    type_dir = os.path.join(result_dir,type_name)
    for model_name in os.listdir(type_dir):
        model_dir = os.path.join(type_dir,model_name)
        for result_path  in [_ for _ in os.listdir(model_dir) if '.csv' in _]:
            if type_name == 'Denovo_Results':
                denovo_uniprot_list.append(os.path.basename(result_path).split('_')[0])
            else:
                hit2lead_uniprot_list.append(os.path.basename(result_path).split(f'_{model_name}')[0])
denovo_uniprot_list = list(set(denovo_uniprot_list))
hit2lead_uniprot_list = list(set(hit2lead_uniprot_list))
print(denovo_uniprot_list,hit2lead_uniprot_list)


['Q92731', 'O43570', 'P00918', 'P00915', 'Q92769', 'P09874'] ['P04629_Sries44458', 'P11309_Sries44702', 'P09874_Sries44306', 'P43490_Sries44327', 'P11309_Sries44180', 'O60885_Sries44384']


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
def replace_model_name(model_name):
    if 'diffSBDD_cond_crossdocked' in model_name:
        return 'DiffSBDD-C'
    elif 'diffSBDD_cond_moad' in model_name:
        return 'DiffSBDD-M'
    elif 'shepherd' in model_name:
        return 'ShEPhERD'
    elif 'DeleteHit2Lead' in model_name:
        return 'Delete'
    else:
        return model_name
record_time_stamp = [1, 10, 100, 1000,2000, 3000,4000,5000,6000,7000,8000,9000,10000, 20000,30000,40000,50000,60000,70000,80000,90000,100000]
X_TICKS = [1, 10, 100, 1000,2000, 3000,4000,5000,6000,7000,8000,9000,10000, 20000,30000,40000,50000,60000,70000,80000,90000,100000]
print(X_TICKS )




def _prepare_subset(df):
    
    if 'mol_idx' not in df.columns:
        df = df.copy()
        df['mol_idx'] = np.arange(len(df))+1
    else:
        if df['mol_idx'].min() != 1:

            df['mol_idx'] = df['mol_idx']+1
    sub = df[df['mol_idx'].isin(record_time_stamp)].copy()
    return sub.sort_values('mol_idx')

def _safe_log(arr):
    arr = np.asarray(arr, dtype=float)
    # log10 for positive values; others become NaN
    with np.errstate(divide='ignore', invalid='ignore'):
        out = np.where(arr > 0, np.log10(arr), np.nan)
    # replace NaN / inf with 0
    out[~np.isfinite(out)] = 0.0
    return out

def _apply_x_ticks(log_x):
    plt.xticks(X_TICKS,fontsize=14)
def _ensure_mol_idx(df):
    return _prepare_subset(df)

def _prepare_x_y(sub, metric, log_x, log_y):
    x = sub['mol_idx'].values
    y = sub[metric].values
    x_plot = np.log10(x) if log_x else x
    y_plot = _safe_log(y) if log_y else y
    return x_plot, y_plot

def _plot_metric_by_uniprot(sampling_record_dict, model_list, save_dir, type_name,
                            metric, ylabel, title_suffix, filename, color_dict,marker,linestyle,
                            log_x=True, log_y=True, apply_x_ticks=True):
    for uniprot_id, model_df_map in sampling_record_dict.items():
        plt.figure(figsize=(8,6))
        for m in model_list:
            if m not in model_df_map:
                continue
            sub = _ensure_mol_idx(model_df_map[m])
            if metric not in sub.columns:
                continue
            x_plot, y_plot = _prepare_x_y(sub, metric, log_x, log_y)
            plt.plot(x_plot, y_plot, 
                     marker=marker, markersize=10,
                     linewidth=2.5, linestyle=linestyle,
                     label=replace_model_name(m), color=color_dict[replace_model_name(m)], alpha=0.8)
        if log_x:
            plt.xlabel('Sampling Numbers (Log10)',fontsize=14)
        else:
            plt.xlabel('Sampling Numbers',fontsize=14)
        if log_y:
            plt.ylabel(f'{ylabel} (Log10)',fontsize=14)
        else:
            plt.ylabel(ylabel)
        plt.title(f'{ylabel} - {uniprot_id} ({title_suffix})', fontsize=14)
        if apply_x_ticks:
            _apply_x_ticks(log_x)
        plt.legend(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.4)
        plt.tight_layout()
        out_dir = os.path.join(save_dir, type_name, uniprot_id)
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(os.path.join(out_dir, filename), dpi=660)
        plt.close()

def plot_scaffold_find_all_models(sampling_record_dict, model_list, save_dir, type_name,
                                    log_x=True, log_y=True):
    _plot_metric_by_uniprot(sampling_record_dict, model_list, save_dir, type_name,
                            metric='scaffold_find_list_filtered_trainset',
                            ylabel='Finded Smiles With Active Scaffold',
                            title_suffix='All Models',
                            filename='plot_smileswith_active_scaffold_find_all_models.svg',
                            log_x=log_x, log_y=log_y, apply_x_ticks=log_x)

def plot_only_scaffold_find_all_models(sampling_record_dict, model_list, save_dir, type_name, color_dict,marker,
                                        log_x=True, log_y=True):
    _plot_metric_by_uniprot(sampling_record_dict, model_list, save_dir, type_name,
                            metric='only_scaffold_find_list_filtered_trainset',
                            ylabel='Finded Unique Active Scaffold',
                            title_suffix='All Models',
                            filename='plot_only_scaffold_find_all_models.svg',
                            color_dict=color_dict,marker=marker,linestyle='-',
                            log_x=log_x, log_y=log_y, apply_x_ticks=log_x)

def plot_smiles_find_all_models(sampling_record_dict, model_list, save_dir, type_name, color_dict,marker,
                                log_x=True, log_y=False):
    _plot_metric_by_uniprot(sampling_record_dict, model_list, save_dir, type_name,
                            metric='smiles_find_list_filtered_trainset',
                            ylabel='Finded Active Smiles',
                            title_suffix='All Models',
                            filename='plot_smiles_find_all_models.svg',
                            color_dict=color_dict,marker=marker,linestyle='--',
                            log_x=log_x, log_y=log_y, apply_x_ticks=log_x)

def plot_log_smiles_find_all_models(sampling_record_dict, model_list, save_dir, type_name,
                                    log_x=True):
    plot_smiles_find_all_models(sampling_record_dict, model_list, save_dir, type_name,
                                log_x=log_x, log_y=True)

def plot_scaffold2smiles_div_scaffold_all_models(sampling_record_dict, model_list, save_dir, type_name,
                                                    log_x=True, log_y=True):
    tmp_dict = {}
    for uniprot_id, model_map in sampling_record_dict.items():
        tmp_dict[uniprot_id] = {}
        for m, df in model_map.items():
            if 'scaffold_find' in df.columns and 'only_scaffold_find' in df.columns:
                sub_df = df.copy()
                denom = sub_df['only_scaffold_find_list_filtered_trainset'].replace(0, np.nan)
                sub_df['ratio_scaffold2smiles_div_scaffold'] = sub_df['scaffold_find_list_filtered_trainset'] / denom
                tmp_dict[uniprot_id][m] = sub_df
    _plot_metric_by_uniprot(tmp_dict, model_list, save_dir, type_name,
                            metric='ratio_scaffold2smiles_div_scaffold',
                            ylabel='Finded Smiles With Active Scaffold / Finded Unique Active Scaffold',
                            title_suffix='All Models',
                            filename='plot_scaffold2smiles_div_scaffold_all_models.svg',
                            log_x=log_x, log_y=log_y, apply_x_ticks=log_y)

[1, 10, 100, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000]


In [ ]:

de_novo_color_list = sns.color_palette("deep", n_colors=10)
hit2lead_color_list = sns.color_palette("Set2", n_colors=7)

de_novo_color_dict = {
    "MolCraft": de_novo_color_list[2],
    "PocketFlow": de_novo_color_list[0],
    "DiffSBDD-C": de_novo_color_list[8],
    "TamGen":de_novo_color_list[6],
}

hit2lead_color_dict = {
    "DiffDec": hit2lead_color_list[1],
    "PGMG": hit2lead_color_list[2],
    "ShapeMol": hit2lead_color_list[3],
    "DiffSBDD-C": hit2lead_color_list[4],
}

from collections import defaultdict
for type_name in ['Denovo_Results','Hit_to_Lead_Results']:
    type_dir = os.path.join(result_dir,type_name)
    if type_name == 'Denovo_Results':
        uniprot_list = denovo_uniprot_list
        color_dict = de_novo_color_dict
        marker = 'o'
    else:
        uniprot_list = hit2lead_uniprot_list
        color_dict = hit2lead_color_dict
        marker = '^'
    print(type_name,uniprot_list)
    for uniprot_id in uniprot_list:
        sampling_record_dict = defaultdict(dict)
        model_list = []
        
        for model_name in os.listdir(type_dir):
            model_list.append(model_name)
            model_dir = os.path.join(type_dir,model_name)
            for result_path  in [_ for _ in os.listdir(model_dir) if '.csv' in _]:
                if uniprot_id in result_path:
                    df = pd.read_csv(os.path.join(model_dir,result_path))
                    sampling_record_dict[uniprot_id][model_name] = df
        os.makedirs(save_dir,exist_ok=True)
        os.makedirs(os.path.join(save_dir,type_name,uniprot_id),exist_ok=True)
        
        plot_only_scaffold_find_all_models(sampling_record_dict,model_list,save_dir,type_name,color_dict,marker,log_x=False,log_y=False)
        plot_smiles_find_all_models(sampling_record_dict,model_list,save_dir,type_name,color_dict,marker,log_x=False,log_y=False)
    

Denovo_Results ['Q92731', 'O43570', 'P00918', 'P00915', 'Q92769', 'P09874']
Hit_to_Lead_Results ['P04629_Sries44458', 'P11309_Sries44702', 'P09874_Sries44306', 'P43490_Sries44327', 'P11309_Sries44180', 'O60885_Sries44384']
